In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import stats
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import MultipleLocator
import pickle
from pprint import pprint as pp
# import seaborn as sns
import sys, os
VICO_loc = '/Users/emigardiner/VICO/pjams-ionization'
Anglada_loc = VICO_loc+'/AngladaRosero2019'
sys.path.append(Anglada_loc)
from def_seds import *

# SOMA data

In [ ]:
# flux scaled for distance
# S_nu d^2 = 
SOMA_dict_inner = {
     b'AFGL_4029':      {'Lbest': 9700,   'Lmin': 3400,   'Lmax': 41000, 'Sd2': 0.5680000000000001, 'color': '#ff7f0e'}, #orange   # S_nu d^2 in mJy
     b'AFGL_437':       {'Lbest': 14000,  'Lmin': 3000,   'Lmax': 93000, 'Sd2': 3.084, 'color': '#2ca02c'},  # green               
     b'Cepheus_A':      {'Lbest': 24000,  'Lmin': 24000,  'Lmax': 99000, 'Sd2': 3.3564999999999996, 'color': '#8c564b'},   # brown  
     b'G35.20-0.74':    {'Lbest': 39000,  'Lmin': 39000,  'Lmax': 84000, 'Sd2': 3.5670800000000007, 'color': '#bcbd22'}, # yellow
     b'G45.47+0.05':    {'Lbest': 460000, 'Lmin': 172000, 'Lmax': 509000, 'Sd2': 6420.96, 'color': '#d62728'}, #red
     b'IRAS_07299-1651':{'Lbest': 20000,  'Lmin': 10000,  'Lmax': 42000, 'Sd2': 3.2457599999999993, 'color': '#0f0bc6'}, #dark blue
     b'IRAS_20126+4104':{'Lbest': 9000,   'Lmin': 9000,   'Lmax': 93000, 'Sd2': 0.15868639999999998, 'color': '#9467bd'}, # purple
     b'NGC_7538_IRS9':  {'Lbest': 37000,  'Lmin': 37000,  'Lmax': 82000, 'Sd2': 2.92136, 'color': '#e377c2'} # pink
    }

SOMA_dict_intermed= {
     b'AFGL_4029': {'Lbest': 9700,  'Lmin': 3400, 'Lmax': 41000, 'Sd2': 1.24, 'color': '#1f77b4'},                 
     b'Cepheus_A':{'Lbest': 24000, 'Lmin': 24000, 'Lmax': 99000, 'Sd2': 4.801999999999999, 'color': '#2ca02c'},        
     b'G35.20-0.74': {'Lbest': 39000, 'Lmin': 39000, 'Lmax': 84000, 'Sd2': 70.18, 'color': '#0f0bc6'},
    }

SOMA_dict_SOMA = {
     b'AFGL_4029': {'Lbest': 9700,  'Lmin': 3400, 'Lmax': 41000, 'Sd2': 1.76, 'color': '#1f77b4'},     # S_nu d^2 in mJy
     b'AFGL_437': {'Lbest': 14000, 'Lmin': 3000, 'Lmax': 93000, 'Sd2': 3.28, 'color': '#ff7f0e'},      
     b'G35.20-0.74': {'Lbest': 39000, 'Lmin': 39000, 'Lmax': 84000, 'Sd2': 70.18, 'color': '#0f0bc6'}, 
     b'G45.47+0.05': {'Lbest': 460000, 'Lmin': 172000, 'Lmax': 509000, 'Sd2': 4092.48, 'color': '#9467bd'},
     b'IRAS_07299-1651': {'Lbest': 20000, 'Lmin': 10000, 'Lmax': 42000, 'Sd2': 4.402943999999999, 'color': '#8c564b'},
     b'IRAS_20126+4104': {'Lbest': 9000, 'Lmin': 9000, 'Lmax': 93000, 'Sd2': 0.9951519999999998, 'color': '#e377c2'}, 
     b'NGC_7538_IRS9': {'Lbest': 37000, 'Lmin': 37000, 'Lmax': 82000, 'Sd2': 35.1125, 'color': '#d62728'}
    }

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#0f0bc6', '#9467bd', 
          '#8c564b', '#e377c2', '#d62728', '', '#17becf']

          #1f77b4

# Ref Data Functions

## ZAMS

In [ ]:
# ZAMS

def draw_ZAMS(ax):
    #pl.loglog(10**np.array(logL_Th), rad_lum_Th,'-k', linewidth=3,label='_nolegend_',alpha=0.8)
    logL_bol_cl, logNe_05_cl, logNe_95_cl = [], [], []

    #cluster_file= 'AngladaData/lbin_cluster_Lbol-Nlym.dat'
    cluster_file= Anglada_loc+'/AngladaData/lbin_cluster_Lbol-Nlym.dat'

    ##Calculating the number of Lyman cont photons
    ##using equation from Monge
    T_e= 1e4    #K
    nu= 6.       #GHz at 5 cm

    ##Reading the input from the table
    for line in open(cluster_file, 'r'):
        l1 = line.split()
        if l1==[]: continue
        skipChars = ['#']
        if line[0] in skipChars: continue
        this_logL_bol_cl= float(l1[0])
        this_logNe_05_cl= float(l1[1])
        this_logNe_95_cl= float(l1[2])

        logL_bol_cl.append( this_logL_bol_cl )
        logNe_05_cl.append( this_logNe_05_cl )
        logNe_95_cl.append( this_logNe_95_cl )

    rad_lum_cesaroni_data=  2.08e-46 * 10**(np.array(logNe_95_cl))* nu**-0.1 * T_e**0.45 
    ax.loglog(10**pl.array(logL_bol_cl), rad_lum_cesaroni_data, '-', linewidth=3, color='olive')


## Tanaka YSO

In [ ]:
def draw_YSO(ax):
        infile_M60 = Anglada_loc+'/AngladaData/mcore60.sigma1.dat'

        M_star_M60, r_star_M60, L_star_M60, T_star_M60, Q_star_M60, rad_lum_star_M60 = \
                pl.loadtxt(infile_M60, unpack=True ,usecols=[0, 1, 2, 3, 4, 5])

        ax.loglog(L_star_M60, rad_lum_star_M60,'-', color= 'teal', linewidth=3,label='_nolegend_',alpha=0.8)

## SOMA sources

In [ ]:

def draw_SOMA(ax, SOMA_dict, debug=False):
    for source in SOMA_dict:
        if(debug): print(source, SOMA_dict.source)
        ax.errorbar(SOMA_dict[source]['Lbest'], SOMA_dict[source]['Sd2'],
                    xerr=[[SOMA_dict[source]['Lbest'] - SOMA_dict[source]['Lmin']], 
                          [SOMA_dict[source]['Lmax'] - SOMA_dict[source]['Lbest']]],
                    ecolor=SOMA_dict[source]['color'], elinewidth=2, capsize=7, capthick=3, alpha=0.25)
        ax.loglog(SOMA_dict[source]['Lbest'], SOMA_dict[source]['Sd2'], 'o',
                  label=source.decode('utf-8'), color=SOMA_dict[source]['color'], 
                  markersize=14,markeredgewidth=1.5, alpha=0.5)


## Shock Data
<!-- 
Years            	1000au   	4000au   	32000au <br /> 
Hi-res ratio <br /> 
9,000 yrs hi-res 	0.004282 	0.005285 	0.191204 <br /> 
21,000 yrs hi-res 	0.000097 	0.000325 	0.129702 <br /> 
39,000 yrs hi-res 	0.001526 	0.001951 	0.070682 <br /> 
54,000 yrs hi-res 	0.103195 	0.211512 	0.709238 <br /> 
<br /> 
Mid-Res ratio <br /> 
9,000 yrs mid-res 	0.022141 	0.022926 	0.201473 <br /> 
21,000 yrs mid-res 	0.013063 	0.013076 	0.077536 <br /> 
39,000 yrs mid-res 	0.026858 	0.031280 	0.290204 <br /> 
54,000 yrs mid-res 	0.067831 	0.132885 	0.377723 <br /> 
<br /> 
Lo-res ratio <br /> 
Snap009_nlr       	0.028766 	0.050783 	0.070038 <br /> 
Snap021_nlr        	0.022532 	0.023125 	0.031393 <br /> 
Snap039_nlr        	0.037043 	0.048523 	0.074351 <br /> 
Snap054_nlr        	0.056095 	0.146190 	0.338637 <br /> 
<br /> 
<br /> 
<br /> 
Years            	1000au   	4000au   	32000au <br /> 
Hi-res filled <br /> 
9,000 yrs hi-res 	0.162599 	0.329726 	281.248527 <br /> 
21,000 yrs hi-res 	0.003545 	0.024687 	107.867878 <br /> 
39,000 yrs hi-res 	0.169222 	0.182894 	32.942426 <br /> 
54,000 yrs hi-res 	19.813620 	46.348223 	620.155483 <br /> 
<br /> 
Mid-Res filled <br /> 
9,000 yrs mid-res 	1.793031 	1.798037 	491.569311 <br /> 
21,000 yrs mid-res 	1.698362 	1.698411 	30.400045 <br /> 
39,000 yrs mid-res 	2.968243 	3.049831 	502.691932 <br /> 
54,000 yrs mid-res 	7.123924 	8.303050 	127.778781 <br /> 
<br /> 
Lo-res filled <br /> 
Snap009_nlr        	5.082482 	5.588887 	18.283755 <br /> 
Snap021_nlr        	4.809765 	4.813022 	13.289503 <br /> 
Snap039_nlr        	7.572746 	7.784907 	16.576450 <br /> 
Snap054_nlr        	8.932021 	13.108063 	21.517012 <br />  -->


In [ ]:
## Load shock data
shock_data= np.load(Anglada_loc+'/AngladaData/shock_flux_arrays.npz')
flux_filled_1000    = shock_data['flux_filled_1000']   
flux_filled_4000    = shock_data['flux_filled_4000']
flux_filled_32000   = shock_data['flux_filled_32000']
flux_filled_1000_lr = shock_data['flux_filled_1000_lr']
flux_filled_4000_lr = shock_data['flux_filled_4000_lr'] 
flux_filled_32000_lr= shock_data['flux_filled_32000_lr']
flux_filled_1000_hr = shock_data['flux_filled_1000_hr']
flux_filled_4000_hr = shock_data['flux_filled_4000_hr'] 
flux_filled_32000_hr= shock_data['flux_filled_32000_hr']
flux_ratio_1000     = shock_data['flux_ratio_1000']     
flux_ratio_4000     = shock_data['flux_ratio_4000']     
flux_ratio_32000    = shock_data['flux_ratio_32000']
flux_ratio_1000_lr  = shock_data['flux_ratio_1000_lr']  
flux_ratio_4000_lr  = shock_data['flux_ratio_4000_lr']  
flux_ratio_32000_lr = shock_data['flux_ratio_32000_lr']
flux_ratio_1000_hr  = shock_data['flux_ratio_1000_hr'] 
flux_ratio_4000_hr  = shock_data['flux_ratio_4000_hr']  
flux_ratio_25000_hr = shock_data['flux_ratio_32000_hr']
shock_data.close()

# Make Figure

In [ ]:
fig = plt.figure(9, figsize=(9,9))
ax = fig.add_axes([.15,.15,.8,.75])


ax.set_ylim(1e-4,1.0e6)
ax.set_xlim(1e-1,1.45e6)

ax.tick_params('both', length=10, width=1, which='major', right=True, top=True, direction='in')
ax.tick_params('both', length=4, width=1, which='minor', right=True, top=True, direction='in')


draw_ZAMS(ax)
draw_YSO(ax)
draw_SOMA(ax, SOMA_dict_inner)
ax.legend()
